In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

In [ ]:
driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")
driver.get('https://understat.com/team/Manchester_United/2018')
"""-----------------------------"""
baseTable = driver.find_element_by_id('team-players')
tableRows = baseTable.find_elements_by_tag_name('tr')
"""------------------------------"""
list = []
data =[]
for i in range(0,len(tableRows)):
    list.append(tableRows[i].text)
    data.append(list[i].split(' '))

In [14]:
data = pd.DataFrame(data[1:],columns=['index',"Name","Surname" ,"Position", "Apps", "Min", "Goals_Scored"
            , "Assists", "Sh90", "KP90", "xG", "xA"
            , "xG90", "xA90",'none','non1'])

In [43]:
    
    data.append([name,Position,Apps,Min,Goals,Assists,Sh90,KP90,xG,xA,xg90,xA90])
df = pd.DataFrame(data,columns=["Name", "Position", "Apps", "Min", "Goals_Scored"
            , "Assists", "Sh90", "KP90", "xG", "xA"
            , "xG90", "xA90"])

In [44]:
df

,Name,Position,Apps,Min,Goals_Scored,Assists,Sh90,KP90,xG,xA,xG90,xA90
0,Romelu Lukaku,F,26,1678,12,0,2.47,0.91,10.01-1.99,1.68+1.68,0.54,0.09
